# realsense depth images

Code snippets to get stuff done with the realsense cam. This repo is not made for colab, it depends on external hardware

## activate right environment
conda activate audiomocap



In [5]:
# First import the library
import pyrealsense2 as rs
# Import Numpy for easy array manipulation
import numpy as np
# Import OpenCV for easy image rendering
import cv2
#imporrt mediapipe for pose detection
import mediapipe as mp
# initialize Pose estimator
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)


# Create a pipeline
pipeline = rs.pipeline()

# Create a config and configure the pipeline to stream
#  different resolutions of color and depth streams
config = rs.config()

# print(dir(config))

# Get device product line for setting a supporting resolution
pipeline_wrapper = rs.pipeline_wrapper(pipeline)
pipeline_profile = config.resolve(pipeline_wrapper)
device = pipeline_profile.get_device()
device_product_line = str(device.get_info(rs.camera_info.product_line))

found_rgb = False
for s in device.sensors:
    if s.get_info(rs.camera_info.name) == 'RGB Camera':
        found_rgb = True
        break
if not found_rgb:
    print("The demo requires Depth camera with Color sensor")
    exit(0)

config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

if device_product_line == 'L500':
    config.enable_stream(rs.stream.color, 960, 540, rs.format.bgr8, 30)
else:
    config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
profile = pipeline.start(config)

# Getting the depth sensor's depth scale (see rs-align example for explanation)
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()
print("Depth Scale is: " , depth_scale)

spatial = rs.spatial_filter()
spatial.set_option(rs.option.holes_fill, 2)
# We will be removing the background of objects more than
#  clipping_distance_in_meters meters away
clipping_distance_in_meters = 1.5 #1 meter
clipping_distance = clipping_distance_in_meters / depth_scale

# Create an align object
# rs.align allows us to perform alignment of depth frames to others frames
# The "align_to" is the stream type to which we plan to align depth frames.
align_to = rs.stream.color
align = rs.align(align_to)


imgNumber = 0

# Streaming loop
try:
    while True:
        # Get frameset of color and depth
        frames = pipeline.wait_for_frames()
        # frames.get_depth_frame() is a 640x360 depth image

        # Align the depth frame to color frame
        aligned_frames = align.process(frames)

        # Get aligned frames
        aligned_depth_frame = aligned_frames.get_depth_frame() # aligned_depth_frame is a 640x480 depth image
        color_frame = aligned_frames.get_color_frame()

        # Validate that both frames are valid
        if not aligned_depth_frame or not color_frame:
            continue

        depth_image = np.asanyarray(aligned_depth_frame.get_data())
        #spatial filters
        
        filtered_depth = np.asanyarray(spatial.process(aligned_depth_frame).get_data())

        color_image = np.asanyarray(color_frame.get_data())
        # images = np.hstack((depth_image,  color_image))

        # Remove background - Set pixels further than clipping_distance to grey
        grey_color = 153
        # depth_image_3d = np.dstack((depth_image,depth_image,depth_image)) #depth image is 1 channel, color is 3 channels
        alpha_image = np.full((filtered_depth.shape[0],filtered_depth.shape[1]), 128, dtype=np.uint8)
        alpha_image = np.where(( filtered_depth> clipping_distance) | (filtered_depth <= 0), 0, 256)
        depth_image_3d = np.dstack((filtered_depth,filtered_depth,filtered_depth)) #depth image is 1 channel, color is 3 channels
        bg_removed = np.where((depth_image_3d > clipping_distance) | (depth_image_3d <= 0), grey_color, color_image)

        # input_image_alphachann = np.full((input_image.shape[0],input_image.shape[1]), 128, dtype=np.uint8)

        bg_alpha= np.dstack((color_image, alpha_image))

        # RGB = cv2.imdecode(bg_removed, -1)
        RGB = bg_removed
        results = pose.process(RGB)
        if (results.pose_landmarks != None):
            print("x =",results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].x)
            print("y = ", results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].y)
            print("z = ",results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].z)

            mp_drawing.draw_landmarks(RGB, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Render images:
        #   depth align to color on left
        #   depth on right
        # depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(filtered_depth, alpha=0.03), cv2.COLORMAP_JET)
        images = np.hstack((bg_removed, depth_colormap))


        

        #save PNG
        # imgNumber = imgNumber+1
        # frameImageFileName = str(f'recording/alpha/image{imgNumber}.png')
        # colorframeImageFileName = str(f'recording/color/image{imgNumber}.png')
        # cv2.imwrite(frameImageFileName, bg_alpha)
        # cv2.imwrite(colorframeImageFileName, color_image)
 

        #show images
        cv2.namedWindow('preproces', cv2.WINDOW_NORMAL)
        # cv2.imshow('preproces',  bg_alpha)
        cv2.namedWindow('Align Example', cv2.WINDOW_NORMAL)
        cv2.imshow('Align Example', images)
        cv2.imshow('Output', RGB)
        key = cv2.waitKey(1)
        # Press esc or 'q' to close the image window
        if key & 0xFF == ord('q') or key == 27:
            cv2.destroyAllWindows()
            break
finally:
    pipeline.stop()

Depth Scale is:  0.0010000000474974513
x = 0.5439984798431396
y =  0.3526594042778015
z =  -0.7583162784576416
x = 0.5384728312492371
y =  0.35239964723587036
z =  -0.6770890355110168
x = 0.5355436205863953
y =  0.35208696126937866
z =  -0.6190120577812195
x = 0.5344245433807373
y =  0.3521285951137543
z =  -0.6500537395477295
x = 0.5341082811355591
y =  0.35198259353637695
z =  -0.666627049446106
x = 0.5341832041740417
y =  0.35216429829597473
z =  -0.6841657161712646
x = 0.5342693328857422
y =  0.3519973158836365
z =  -0.6796387434005737
x = 0.5345053672790527
y =  0.3522065579891205
z =  -0.7401502728462219
x = 0.5371115207672119
y =  0.3515893816947937
z =  -0.778457522392273
x = 0.5383902192115784
y =  0.35067763924598694
z =  -0.7614193558692932
x = 0.5384083986282349
y =  0.3506860136985779
z =  -0.7544971704483032
x = 0.5387181043624878
y =  0.3506668210029602
z =  -0.738959789276123
x = 0.5386316776275635
y =  0.3507719337940216
z =  -0.7558912634849548
x = 0.5395983457565308
